# Pre-processing of GDMP 1km  time-series on ALBEDO  raster dataset for the uploaded to JEDI

The following notbooks set up to help raster-data-processing for the uploaded to JEDI
https://jedi.discomap.eea.europa.eu/Search

Requirements for raster upload to JEDI: 
- pixel sizes in (10m, 100, 1000m or 10 000m) 
- projection: LAEA -EPSG3035 
- datatype: INT8,UINT8,INT16 ant UINT16.
- pixel nbit proberty:  NO nbits--> remove NBITS 
- Compression:  -co COMPRESS=JPEG -co TILED=YES
- format: *.tif


Every raster in the zip will be added as an attribute to the dimension.
You can exclude values inside the JEDI website under the field options. Provide a value seperated list of all values you wish to exclude.
The exclude will only work for INT8,UINT8,INT16 ant UINT16.

So, there is a workaround in place:
1. Create a zip with the same tifs but these must be clipped to a small area.
This way the uploaded zip is small but enough to populate the schema in the db.
2. When the dim is created (but not completed) send us the path to the real zip file and we will take care of it



## (Tool 1 -A) small subset to prepare the jedi upload *using a small clip


L:\f02_data\GDMP\input\GDMP_100m

In [46]:
# tool1 layerstack to single band
import os, sys 
#import arcpy
#from arcpy.sa import *

from osgeo import gdal
from osgeo import gdalconst
from osgeo import osr


from pathlib import Path
import os
import sys
from datetime import datetime
import multiprocessing
import glob
import zipfile
import shutil  ## for zipping

#L:\f02_data\GDMP\input\GDMP_100m
# set data folder
#sys.path.append(Path(os.getcwd()).parent.joinpath('src').as_posix())
#sys.path.append(Path(os.getcwd()).joinpath('src').as_posix())
#from constants import dir_signature

###################################################################################################################################### SET
#base_path = os.path.join(dir_signature, 'GDMP')
#base_path_input = os.path.join(base_path, 'input','GDMP_100m') # , 'SOC_g_m2_90-21')   ###inputs
#outzip = OutRas+'zip'
                  #N:\C2307_Copernicus\T25_Mapping Carbon Pools\output\GDMP_1km_for_JEDI\input_gdmp_1km_time_series
input_folder = r'N:\C2307_Copernicus\T25_Mapping Carbon Pools\output\GDMP_1km_for_JEDI\input_gdmp_1km_time_series'
output_folder = r'N:\C2307_Copernicus\T25_Mapping Carbon Pools\output\GDMP_1km_for_JEDI\output_gdmp_1km_time_series'



zip_folder = output_folder
raster_folder_with_final_raster_datasets = output_folder

#aoi_folder = r'L:\f02_data\GDMP\input\GDMP_100m\aoi'
aoi_folder =r'N:\C2307_Copernicus\T25_Mapping Carbon Pools\output\GDMP_1km_for_JEDI\output_gdmp_1km_time_series\aoi'
###############################################output info:Start 
pixel_type_info ='16_BIT_SIGNED'
format_info = 'COG'
NoData_info = -999
###############################################output info:End

# To allow overwriting outputs change overwriteOutput option to True.
#rcpy.env.overwriteOutput = True
#first_year = 1990  # to put the year into the raster name 
###################################################################################################################################### SET
print ("START.................")

with os.scandir(input_folder) as p:
    for raster in p:
        if  raster.name.endswith(".tif"): # Only show the files whose name ends with dot tif.
            raster_file = os.path.join(input_folder, raster.name) # Get the full path of the raster, you'll need it in order to open the raster file and access its data. 
            #raster = gdal.Open(raster_file) # Open the current raster file.
            raster_name = os.path.join(raster.name)
            raster_output =output_folder+"\\"+raster_name.rsplit('.', maxsplit=1)[0]+'_clip.tif'

            print (raster_name)
          
            extent = (3015500,2933800,4014700,4071300)
            

            upper_left_x = 699934.584491
            upper_left_y = 6169364.0093
            lower_right_x = 700160.946739
            lower_right_y = 6168703.00544
            window = (upper_left_x,upper_left_y,lower_right_x,lower_right_y)
            window = (3015500,2933800,4014700,4071300)
            gdal.Translate(raster_output, raster_file, projWin = window)
         


            print ("next................................")



print ("done")

print (raster_output)

START.................
GDMP_1999_pv_1000m_EPSG3035.tif
next................................
GDMP_2000_pv_1000m_EPSG3035.tif
next................................
GDMP_2001_pv_1000m_EPSG3035.tif
next................................
GDMP_2002_pv_1000m_EPSG3035.tif
next................................
GDMP_2003_pv_1000m_EPSG3035.tif
next................................
GDMP_2004_pv_1000m_EPSG3035.tif
next................................
GDMP_2005_pv_1000m_EPSG3035.tif
next................................
GDMP_2006_pv_1000m_EPSG3035.tif
next................................
GDMP_2007_pv_1000m_EPSG3035.tif
next................................
GDMP_2008_pv_1000m_EPSG3035.tif
next................................
GDMP_2009_pv_1000m_EPSG3035.tif
next................................
GDMP_2010_pv_1000m_EPSG3035.tif
next................................
GDMP_2011_pv_1000m_EPSG3035.tif
next................................
GDMP_2012_pv_1000m_EPSG3035.tif
next................................
GDMP_2013_p

In [ ]:
print ()

## (Tool 1 -B) FULL production of JEDI upload: